# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))


# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))


# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))


# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

Epoch 137/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [===========

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [10]:
# Train the model
normalized_input_model.fit(X_train, y_train, batch_size=32, epochs=150, validation_data=(X_val, y_val))# Train the model

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [11]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [12]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, batch_size=32, epochs=150, validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.4828 - mse: 0.4828 - val_loss: 0.2014 - val_mse: 0.2014
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2526 - mse: 0.2526 - val_loss: 0.1710 - val_mse: 0.1710
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2069 - mse: 0.2069 - val_loss: 0.1524 - val_mse: 0.1524
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1720 - mse: 0.1720 - val_loss: 0.1439 - val_mse: 0.1439
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1543 - mse: 0.1543 - val_loss: 0.1359 - val_mse: 0.1359
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1329 - mse: 0.1329 - val_loss: 0.1344 - val_mse: 0.1344
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1295 - mse: 0.1295 - val_loss: 0.1328 - val_mse: 0.1328
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1165 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0265 - mse: 0.0265 - val_loss: 0.1351 - val_mse: 0.1351
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.1347 - val_mse: 0.1347
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0261 - mse: 0.0261 - val_loss: 0.1427 - val_mse: 0.1427
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.1346 - val_mse: 0.1346
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0257 - mse: 0.0257 - val_loss: 0.1355 - val_mse: 0.1355
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0254 - mse: 0.0254 - val_loss: 0.1328 - val_mse: 0.1328
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.1319 - val_mse: 0.1319
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0242 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.1374 - val_mse: 0.1374
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1374 - val_mse: 0.1374
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0130 - mse: 0.0130 - val_loss: 0.1378 - val_mse: 0.1378
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1379 - val_mse: 0.1379
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1375 - val_mse: 0.1375
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.1390 - val_mse: 0.1390
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0125 - mse: 0.0125 - val_loss: 0.1383 - val_mse: 0.1383
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [13]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)


33/33 [==============================] - 0s 1ms/step - loss: 0.0095 - mse: 0.0095


[0.009458012878894806, 0.009458012878894806]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [14]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 1ms/step - loss: 0.1392 - mse: 0.1392


[0.1391812264919281, 0.1391812264919281]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [15]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

29317.15374391067

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [16]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))


# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', loss='mse', metrics=['mse'])

# Train the model
he_model.fit(X_train, y_train_scaled, batch_size=32, epochs=150, validation_data=(X_val, y_val_scaled))


Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.4849 - mse: 0.4849 - val_loss: 0.2385 - val_mse: 0.2385
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2380 - mse: 0.2380 - val_loss: 0.1753 - val_mse: 0.1753
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1876 - mse: 0.1876 - val_loss: 0.1577 - val_mse: 0.1577
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1650 - mse: 0.1650 - val_loss: 0.1519 - val_mse: 0.1519
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1465 - mse: 0.1465 - val_loss: 0.1450 - val_mse: 0.1450
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1305 - mse: 0.1305 - val_loss: 0.1351 - val_mse: 0.1351
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1236 - mse: 0.1236 - val_loss: 0.1388 - val_mse: 0.1388
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1143 - m

Epoch 63/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0242 - mse: 0.0242 - val_loss: 0.1230 - val_mse: 0.1230
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1239 - val_mse: 0.1239
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1250 - val_mse: 0.1250
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1243 - val_mse: 0.1243
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1255 - val_mse: 0.1255
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1325 - val_mse: 0.1325
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.1327 - val_mse: 0.1327
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.1327 - val_mse: 0.1327
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.1324 - val_mse: 0.1324
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1334 - val_mse: 0.1334
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1342 - val_mse: 0.1342
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.1336 - val_mse: 0.1336
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 803us/step - loss: 0.0090 - mse: 0.0090


[0.009040533564984798, 0.009040533564984798]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [18]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1342 - mse: 0.1342


[0.13422344624996185, 0.13422344624996185]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [19]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', loss='mse', metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 8ms/step - loss: 0.4030 - mse: 0.4030 - val_loss: 0.1722 - val_mse: 0.1722
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2137 - mse: 0.2137 - val_loss: 0.1422 - val_mse: 0.1422
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1752 - mse: 0.1752 - val_loss: 0.1249 - val_mse: 0.1249
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1459 - mse: 0.1459 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1390 - mse: 0.1390 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1266 - mse: 0.1266 - val_loss: 0.1049 - val_mse: 0.1049
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1159 - mse: 0.1159 - val_loss: 0.1033 - val_mse: 0.1033
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1072 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0213 - mse: 0.0213 - val_loss: 0.0966 - val_mse: 0.0966
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.0932 - val_mse: 0.0932
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0203 - mse: 0.0203 - val_loss: 0.0952 - val_mse: 0.0952
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0203 - mse: 0.0203 - val_loss: 0.0933 - val_mse: 0.0933
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0202 - mse: 0.0202 - val_loss: 0.0929 - val_mse: 0.0929
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0201 - mse: 0.0201 - val_loss: 0.0932 - val_mse: 0.0932
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0198 - mse: 0.0198 - val_loss: 0.0950 - val_mse: 0.0950
Epoch 70/150
33/33 [==============================] - 0s 991us/step - loss: 0.0190 - mse:

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.0920 - val_mse: 0.0920
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.0913 - val_mse: 0.0913
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.0911 - val_mse: 0.0911
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.0906 - val_mse: 0.0906
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.0912 - val_mse: 0.0912
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.0905 - val_mse: 0.0905
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.0912 - val_mse: 0.0912
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0073 - mse: 0.0073


[0.007341817952692509, 0.007341817952692509]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.0907 - mse: 0.0907


[0.09073077887296677, 0.09073077887296677]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [22]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 6ms/step - loss: 0.3381 - mse: 0.3381 - val_loss: 0.1433 - val_mse: 0.1433
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1811 - mse: 0.1811 - val_loss: 0.1200 - val_mse: 0.1200
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1549 - mse: 0.1549 - val_loss: 0.1239 - val_mse: 0.1239
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1162 - mse: 0.1162 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0948 - mse: 0.0948 - val_loss: 0.1176 - val_mse: 0.1176
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0841 - mse: 0.0841 - val_loss: 0.1454 - val_mse: 0.1454
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0662 - mse: 0.0662 - val_loss: 0.1328 - val_mse: 0.1328
Epoch 8/150
33/33 [==============================] - ETA: 0s - loss: 0.0302 - mse: 

Epoch 63/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.1364 - val_mse: 0.1364
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1203 - val_mse: 0.1203
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.1301 - val_mse: 0.1301
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1098 - val_mse: 0.1098
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1621 - val_mse: 0.1621
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.1153 - val_mse: 0.1153
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0080 - mse: 0.0080 - val_loss: 0.1389 - val_mse: 0.1389
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.1243 - val_mse: 0.1243
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1342 - val_mse: 0.1342
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.1254 - val_mse: 0.1254
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0051 - mse: 0.0051 - val_loss: 0.1304 - val_mse: 0.1304
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.1117 - val_mse: 0.1117
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.1148 - val_mse: 0.1148
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0031 - mse: 0.0031


[0.003118982771411538, 0.003118982771411538]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [25]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 1ms/step - loss: 0.1225 - mse: 0.1225


[0.12246917188167572, 0.12246917188167572]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [26]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer='Adam', loss='mse', metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: 0.4068 - mse: 0.4068 - val_loss: 0.1572 - val_mse: 0.1572
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1552 - mse: 0.1552 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1064 - mse: 0.1064 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0789 - mse: 0.0789 - val_loss: 0.0970 - val_mse: 0.0970
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0577 - mse: 0.0577 - val_loss: 0.1121 - val_mse: 0.1121
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0473 - mse: 0.0473 - val_loss: 0.1118 - val_mse: 0.1118
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0368 - mse: 0.0368 - val_loss: 0.1096 - val_mse: 0.1096
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0297 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1080 - val_mse: 0.1080
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0146 - mse: 0.0146 - val_loss: 0.1093 - val_mse: 0.1093
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1022 - val_mse: 0.1022
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.1047 - val_mse: 0.1047
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0132 - mse: 0.0132 - val_loss: 0.1007 - val_mse: 0.1007
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0082 - mse: 0.0082 - val_loss: 0.1078 - val_mse: 0.1078
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0065 - mse: 0.0065 - val_loss: 0.0963 - val_mse: 0.0963
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0112 - mse: 0

33/33 [==============================] - 0s 2ms/step - loss: 0.0143 - mse: 0.0143 - val_loss: 0.0990 - val_mse: 0.0990
Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.0936 - val_mse: 0.0936
Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0997 - val_mse: 0.0997
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.0920 - val_mse: 0.0920
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0965 - val_mse: 0.0965
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0976 - val_mse: 0.0976
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0027 -

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)


33/33 [==============================] - 0s 937us/step - loss: 0.0016 - mse: 0.0016


[0.0015899050049483776, 0.0015899050049483776]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [28]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 826us/step - loss: 0.0984 - mse: 0.0984


[0.09838388115167618, 0.09838388115167618]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [29]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)


5/5 [==============================] - 0s 2ms/step - loss: 0.1783 - mse: 0.1783


[0.17833800613880157, 0.17833800613880157]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [30]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

33185.8924171614

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.